In [ ]:
%pip install ipykernel
%pip install pymongo sentence-transformers scikit-learn numpy
%pip install patool pyunpack

print("Todas las dependencias instaladas correctamente")

import pymongo
from sentence_transformers import SentenceTransformer
import sklearn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
import subprocess

: 

In [ ]:
import os
import json
import hashlib


CARPETA_DISCURSOS = "DiscursosOriginales"
ARCHIVO_SALIDA = "discursos-final-202373580-9-202473508-1.json"

model = SentenceTransformer("all-MiniLM-L6-v2")

def calcular_sha256(texto: str) -> str:
    h = hashlib.sha256()
    h.update(texto.encode("utf-8"))
    return h.hexdigest()

def procesar_discursos():
    documentos = []

    for nombre in sorted(os.listdir(CARPETA_DISCURSOS)):
        if not nombre.lower().endswith(".txt"):
            continue

        ruta_archivo = os.path.join(CARPETA_DISCURSOS, nombre)

        with open(ruta_archivo, "r", encoding="utf-8") as f:
            texto = f.read().strip()

        _id = calcular_sha256(texto)
        embedding = model.encode(texto).tolist()

        doc = {
            "_id": _id,
            "texto": texto,
            "embedding": embedding
        }

        documentos.append(doc)
        print(f"Procesado {nombre}")

    # Guardar como JSON en forma de ARRAY
    with open(ARCHIVO_SALIDA, "w", encoding="utf-8") as out:
        json.dump(documentos, out, ensure_ascii=False, indent=2)

    print(f"\nSe guardaron {len(documentos)} documentos en {ARCHIVO_SALIDA}")

if __name__ == "__main__":
    procesar_discursos()


In [ ]:
from pymongo import MongoClient
import json

client = MongoClient(
    "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set&readPreference=primary&ssl=false"
)

db = client["Politica"]
coleccion = db["Discursos"]

print("Conectado a MongoDB Replica Set")
print(f"Nodos disponibles: {client.nodes}")

In [ ]:
from pymongo.errors import BulkWriteError

with open("discursos-final-202373580-9-202473508-1.json", "r", encoding="utf-8") as f:
    documentos = json.load(f)

try:
    resultado = coleccion.insert_many(documentos, ordered=False)
except BulkWriteError:
    pass

In [ ]:
def buscar_discursos_similares(consulta_texto, top_k=5):

    embedding_consulta = model.encode(consulta_texto).reshape(1, -1)
    
    documentos = list(coleccion.find({}))
    

    resultados = []
    for doc in documentos:
        embedding_doc = np.array(doc['embedding']).reshape(1, -1)
        similitud = cosine_similarity(embedding_consulta, embedding_doc)[0][0]
        
        resultados.append({
            '_id': doc['_id'],
            'texto': doc['texto'],
            'similitud': similitud
        })
    

    resultados_ordenados = sorted(resultados, key=lambda x: x['similitud'], reverse=True)[:top_k]
    
    return resultados_ordenados

consulta_ejemplo = "discurso sobre la democracia y la libertad"
resultados = buscar_discursos_similares(consulta_ejemplo)

print("Top 5 discursos más similares:")
for i, resultado in enumerate(resultados, 1):
    print(f"\n--- Resultado {i} ---")
    print(f"Similitud: {resultado['similitud']:.4f}")
    print(f"Texto: {resultado['texto']}")
    print(f"ID: {resultado['_id']}")

In [ ]:
print("=== PRUEBA DE BÚSQUEDA SEMÁNTICA ===")
consulta_ejemplo = "discurso sobre la democracia y la libertad"
resultados = buscar_discursos_similares(consulta_ejemplo)

print(f"Consulta: '{consulta_ejemplo}'")
print("\nTop 5 discursos más similares:")
for i, resultado in enumerate(resultados, 1):
    print(f"\n--- Resultado {i} (Similitud: {resultado['similitud']:.4f}) ---")
    print(f"Texto: {resultado['texto']}")

In [ ]:
count_inicial = coleccion.count_documents({})
print(f"Documentos iniciales en colección: {count_inicial}")

doc_prueba = {
    "_id": "test_consistencia_123",
    "texto": "Documento de prueba para consistencia",
    "embedding": model.encode("Documento de prueba para consistencia").tolist()
}

resultado_insercion = coleccion.insert_one(doc_prueba)
print(f"Documento de prueba insertado: {resultado_insercion.inserted_id}")

encontrado = coleccion.find_one({"_id": "test_consistencia_123"})
print(f"Documento encontrado después de inserción: {encontrado is not None}")

count_despues_insercion = coleccion.count_documents({})
print(f"Documentos después de inserción: {count_despues_insercion}")

resultado_eliminacion = coleccion.delete_one({"_id": "test_consistencia_123"})
print(f"Documentos eliminados: {resultado_eliminacion.deleted_count}")

encontrado = coleccion.find_one({"_id": "test_consistencia_123"})
print(f"Documento encontrado después de eliminación: {encontrado is not None}")

count_final = coleccion.count_documents({})
print(f"Documentos finales: {count_final}")

print("Consistencia verificada: Los datos se mantienen consistentes en todas las operaciones")

In [ ]:
print("\n=== DEMOSTRACIÓN DE ALTA DISPONIBILIDAD ===")

try:

    status = client.admin.command('replSetGetStatus')
    print("Estado del Replica Set:")
    for member in status['members']:
        estado = f"  - {member['name']}: {member['stateStr']} (health: {member['health']})"
        if member['stateStr'] == 'PRIMARY':
            estado += " ← PRIMARIO"
        print(estado)
    

    print(f"\nNodo primario actual: {client.primary}")
    print(f"Hosts conectados: {client.nodes}")
    
    print("\nPara demostrar alta disponibilidad:")
    print("1. Detén manualmente el contenedor del nodo primario (ej: docker stop mongo1)")
    print("2. Ejecuta nuevamente una consulta de búsqueda")
    print("3. El sistema debería seguir funcionando con otro nodo como primario")
    

    print("\nRealizando consulta de prueba...")
    resultado_prueba = coleccion.find_one({})
    print("Consulta ejecutada exitosamente durante operación normal")
    
except Exception as e:
    print(f"Error en la demostración: {e}")